In [19]:
import os
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain.chains import GraphCypherQAChain

In [16]:
from dotenv import load_dotenv

load_dotenv()

URI = os.environ.get("NEO4J_URI")
USERNAME = os.environ.get("NEO4J_USERNAME")
PASSWORD = os.environ.get("NEO4J_PASSWORD")
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")

In [7]:
graph = Neo4jGraph(url=URI, username=USERNAME, password=PASSWORD)

In [10]:
print(graph.schema)

Node properties:
Committee {id: STRING, full_name: STRING, highlight_reel_count: INTEGER, bio: STRING, num_followers: INTEGER, num_posts: INTEGER, category: STRING, num_following: INTEGER, username: STRING}
Follower {id: STRING, username: STRING}
Post {id: STRING, comments: INTEGER, caption: STRING, type: STRING, time_taken: STRING, likes: INTEGER, num_children: INTEGER, views: INTEGER}
Relationship properties:

The relationships:
(:Committee)-[:POSTED]->(:Post)
(:Committee)-[:FOLLOWS]->(:Committee)
(:Follower)-[:FOLLOWS]->(:Committee)


In [26]:
llm=ChatGroq(groq_api_key=GROQ_API_KEY, model_name="gemma2-9b-it")

In [27]:
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True)

In [28]:
response=chain.invoke({"query":"Give me information about djcsi committee account."})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Committee {username: "djcsi"}) RETURN c 

Full Context:
[{'c': {'full_name': 'CSI-DJSCE', 'highlight_reel_count': 24, 'bio': 'Student Chapter of Computer Society of India \nD.J. Sanghvi College of Engineering', 'num_followers': 1899, 'num_posts': 380, 'id': '1507035408', 'category': 'Community', 'num_following': 30, 'username': 'djcsi'}}]

> Finished chain.


In [30]:
print(response['result'])

The djcsi committee account is the Student Chapter of Computer Society of India at D.J. Sanghvi College of Engineering.  They have 1899 followers and have posted 380 times.  



In [37]:
response=chain.invoke({"query":"Please provide me with the usernames of the ten djs.sigai committee account followers."})
print(response['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Committee {username: 'djs.sigai'})<-[:FOLLOWS]-(f:Follower)
RETURN f.username LIMIT 10  

Full Context:
[{'f.username': 'kartikk14_'}, {'f.username': 'therudra1064'}, {'f.username': 'devvvv_2607'}, {'f.username': 'mithilesh.shirawle08'}, {'f.username': 'shubham.khichaa'}, {'f.username': '_vansh_mandot_'}, {'f.username': 'mahekkk_02'}, {'f.username': 'parth.sangani_'}, {'f.username': 'nabhijkharade'}, {'f.username': 'kajalsandeep'}]

> Finished chain.
kartikk14_, therudra1064, devvvv_2607, mithilesh.shirawle08, shubham.khichaa, _vansh_mandot_, mahekkk_02, parth.sangani_, nabhijkharade, kajalsandeep 



In [56]:
response=chain.invoke({"query":"Give count of different type of post made by djs.s4ds committee."})
print(response['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Committee {username:'djs.s4ds'})-[:POSTED]->(p:Post)
RETURN DISTINCT p.type , count(p)  
AS post_type_count
ORDER BY post_type_count DESC

Full Context:
[{'p.type': 'GraphSidecar', 'post_type_count': 6}, {'p.type': 'GraphVideo', 'post_type_count': 3}, {'p.type': 'GraphImage', 'post_type_count': 3}]

> Finished chain.
GraphSidecar has 6 posts, GraphVideo has 3 posts, and GraphImage has 3 posts.  

